# Transformer

In [79]:
import os
os.environ["KERAS_BACKEND"] = "jax"
import keras

filename = keras.utils.get_file(
    origin=(
        "https://storage.googleapis.com/download.tensorflow.org/"
        "data/shakespeare.txt"
    ),
)
shakespeare = open(filename, "r").read()

In [80]:
shakespeare[:250]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n'

In [81]:
import tensorflow as tf

sequence_length = 100

def split_input(input, sequence_length):
    for i in range(0, len(input), sequence_length):
        yield input[i : i + sequence_length]

features = list(split_input(shakespeare[:-1], sequence_length))
labels = list(split_input(shakespeare[1:], sequence_length))
dataset = tf.data.Dataset.from_tensor_slices((features, labels))

In [82]:
x, y = next(dataset.as_numpy_iterator())

In [83]:
x[:50], y[:50]

(b'First Citizen:\nBefore we proceed any further, hear',
 b'irst Citizen:\nBefore we proceed any further, hear ')

In [84]:
from keras import layers

tokenizer = layers.TextVectorization(
    standardize=None,
    split=lambda x: tf.strings.unicode_split(x, 'UTF-8'),
    output_sequence_length=sequence_length,
)

tokenizer.adapt(dataset.map(lambda text, labels: text))

In [85]:
vocabulary_size = tokenizer.vocabulary_size()
vocabulary_size

67

In [86]:
dataset = dataset.map(
    lambda features, labels: (tokenizer(features), tokenizer(labels)),
    num_parallel_calls=8,
)
training_data = dataset.shuffle(10_000).batch(64).cache()

In [87]:
embedding_dim = 256
hidden_dim = 1024

inputs = layers.Input(shape=(sequence_length,), dtype=tf.int32, name="token_ids")
x = layers.Embedding(vocabulary_size, embedding_dim)(inputs)
x = layers.GRU(hidden_dim, return_sequences=True)(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(vocabulary_size, activation="softmax")(x)
model = keras.Model(inputs, outputs, name="shakespeare")

In [88]:
model.summary(line_length=80)

Model: "shakespeare"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                      ┃ Output Shape             ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ token_ids (InputLayer)            │ (None, 100)              │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ embedding_10 (Embedding)          │ (None, 100, 256)         │        17,152 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ gru_10 (GRU)                      │ (None, 100, 1024)        │     3,938,304 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ dropout_4 (Dropout)               │ (None, 100, 1024)        │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ dense_10 (Dense)                  │ (None, 100, 67)          │        68,675 │
└───────────────────────────────────┴──────────────────────────┴───────────────┘

 Total params: 4,024,131 (15.35 MB)

 Trainable params: 4,024,131 (15.35 MB)

 Non-trainable params: 0 (0.00 B)

In [89]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
model.fit(training_data, epochs=20)

Epoch 1/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 3.0603 - sparse_categorical_accuracy: 0.2360
Epoch 2/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1.9434 - sparse_categorical_accuracy: 0.4285
Epoch 3/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1.6696 - sparse_categorical_accuracy: 0.5024
Epoch 4/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1.5310 - sparse_categorical_accuracy: 0.5390
Epoch 5/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1.4484 - sparse_categorical_accuracy: 0.5602
Epoch 6/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1.3893 - sparse_categorical_accuracy: 0.5747
Epoch 7/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1.3430 - sparse_categorical_accuracy: 0.5863
Epoch 8/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1.3011 - sparse_categorical_accuracy: 0.5969
Epoch 9/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 1.2627 - sparse_categorical_accuracy: 0.6069
Epoch 10/20
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 11

In [90]:
inputs = keras.Input(shape=(1,), dtype="int", name="token_ids")
input_state = keras.Input(shape=(hidden_dim,), name="state")

x = layers.Embedding(vocabulary_size, embedding_dim)(inputs)
x, output_state = layers.GRU(hidden_dim, return_state=True)(
    x, initial_state=input_state
)
outputs = layers.Dense(vocabulary_size, activation="softmax")(x)
generation_model = keras.Model(
    inputs=(inputs, input_state),
    outputs=(outputs, output_state),
)
generation_model.set_weights(model.get_weights())

In [91]:
tokens = tokenizer.get_vocabulary()
token_ids = range(vocabulary_size)
char_to_id = dict(zip(tokens, token_ids))
id_to_char = dict(zip(token_ids, tokens))

prompt = """
KING RICHARD III:
"""

In [94]:
input_ids = [char_to_id[c] for c in prompt]
state = keras.ops.zeros(shape=(1, hidden_dim))
for token_id in input_ids:
    inputs = keras.ops.expand_dims([token_id], axis=0)
    predictions, state = generation_model.predict((inputs, state), verbose=0)

In [95]:
import numpy as np

generated_ids = []
max_length = 250
for i in range(max_length):
    next_char = int(np.argmax(predictions, axis=-1)[0])
    generated_ids.append(next_char)
    inputs = keras.ops.expand_dims([next_char], axis=0)
    predictions, state = generation_model.predict((inputs, state), verbose=0)

NameError: name 'predictions' is not defined